# Tuan YU
# Word2vec & Semantic Similarity Assignment

## 1. For any one selected article title from the dataset, finds 10 most similar titles (excluding the selected article title) based on Word2Vec similarity, prints those titles and their similarity scores in a descending order of similarity scores.

In [1]:
import gensim, operator
from scipy import spatial
import numpy as np
from gensim.models import KeyedVectors

## Import the data and select a title

In [3]:
import pandas as pd
json = pd.read_json("webhose_netflix-Copy1.json",lines=True)

In [6]:
documents=[]
for i in range(len(json)):
    documents.append(json.title[i])
len(documents)

25288

## Deduplicate the dataset by title

In [11]:
deduptitle=[]
dedupdocu=[]
for rows in json.itertuples():
    if len(rows.title)!=0:
        if rows.title not in list(set(deduptitle)):
            r=rows
            dedupdocu.append(r)
            deduptitle.append(rows.title)

len(dedupdocu)

19513

In [12]:
dedupdocu=pd.DataFrame(dedupdocu)
dedupdocu['title'].value_counts

<bound method IndexOpsMixin.value_counts of 0        13 Reasons Why: The popular Netflix show's cre...
1        Judge gives control of 'Tiger King' Joe Exotic...
2        A TV reboot of Bong Joon-ho's acclaimed film S...
3        2-Pack: Ideaworks Mosquito Killer Lamps (batte...
4        Already-Obese Average Americans Have Drunk & E...
                               ...                        
19508    Jerry Seinfeld channels James Bond in Netflix ...
19509    Why you should embrace the quarantine boredom:...
19510    SHE-RA E AS PRINCESAS DO PODER: Trailer Final ...
19511        7 Netflix shows canceled way too soon, ranked
19512           New on Netflix, HBO, Hulu, Amazon May 2020
Name: title, Length: 19513, dtype: object>

## Import the word2vec pre-trained model

In [4]:
# location of the downloaded binary
filepath = 'GoogleNews-vectors-negative300.bin'

In [5]:
model = KeyedVectors.load_word2vec_format(filepath, binary=True)

##  Prepare the unique title list

In [17]:
deduptitle=list(set(deduptitle))
titles=[]
for t in deduptitle:
    titles.append({'text':t})

In [18]:
titles[0:10]

[{'text': "Martin Scorsese's next film to feature Leonardo DiCaprio, Robert De Niro"},
 {'text': 'What To Do If Your Employer Has Stopped Matching Your 401k Contributions'},
 {'text': 'Reflecting on Luke Cage, the Pioneering Black Superhero, and His Lasting Legacy'},
 {'text': 'HBO is taking on Netflix with human curation instead of relying on algorithms'},
 {'text': '20:15 | Das Erste | Naturwunder Okawango'},
 {'text': 'Anurag Kashyap’s Choked to Launch in June on Netflix – The News Of India'},
 {'text': "Despite Trump's Demand, Not All Houses Of Worship Reopen"},
 {'text': 'Learn how to decorate cakes like a pro'},
 {'text': '’13 Reasons Why’ Sets Final Season Premiere Date | Decider'},
 {'text': 'Netflix announces the launch date for Undercover season 2.'}]

## Rank texts based on similiarly

In [21]:
# Ensure the tokens are included in the vocabulary
import re
def clean(text):
    text = re.sub(r'[^\w\s]', '',text)
    text = [word for word in text.split() if word in model.key_to_index]
    return text

In [22]:
input1 = clean(titles[0]['text'])
input1

['Martin',
 'Scorseses',
 'next',
 'film',
 'feature',
 'Leonardo',
 'DiCaprio',
 'Robert',
 'De',
 'Niro']

In [23]:
for t in titles:
    input2 = clean(t['text'])
#some of the titles are empty after cleaning
    if len(input2)!= 0:
        t['sim'] = model.n_similarity(input1, input2)

In [24]:
similar=[]
for t in titles:
    if len(clean(t['text']))!=0:
        similar.append(t)
similar[0:10]

[{'text': "Martin Scorsese's next film to feature Leonardo DiCaprio, Robert De Niro",
  'sim': 1.0},
 {'text': 'What To Do If Your Employer Has Stopped Matching Your 401k Contributions',
  'sim': 0.19035427},
 {'text': 'Reflecting on Luke Cage, the Pioneering Black Superhero, and His Lasting Legacy',
  'sim': 0.4287738},
 {'text': 'HBO is taking on Netflix with human curation instead of relying on algorithms',
  'sim': 0.30790216},
 {'text': '20:15 | Das Erste | Naturwunder Okawango', 'sim': 0.18395117},
 {'text': 'Anurag Kashyap’s Choked to Launch in June on Netflix – The News Of India',
  'sim': 0.30300277},
 {'text': "Despite Trump's Demand, Not All Houses Of Worship Reopen",
  'sim': 0.14312756},
 {'text': 'Learn how to decorate cakes like a pro', 'sim': 0.22588474},
 {'text': '’13 Reasons Why’ Sets Final Season Premiere Date | Decider',
  'sim': 0.23171097},
 {'text': 'Netflix announces the launch date for Undercover season 2.',
  'sim': 0.32902905}]

## Sorting using the built-in **sorted** function

In [25]:
similar=sorted(similar, key=lambda s: s['sim'],reverse=True)
#exclude the similar[0] which should be the original title
for s in similar[1:11]:
    print(similar.index(s),'\n',s['sim'],'\n',s['text'])

1 
 0.9616291 
 Leonardo DiCaprio, Robert De Niro to star in Martin Scorsese's next film
2 
 0.8403441 
 Apple seals pact for Martin Scorsese's $200 million film starring Leonardo DiCaprio, De Niro
3 
 0.80540144 
 Apple secures new Scorsese film starring DiCaprio, De Niro
4 
 0.7868109 
 Apple secures deal for Scorsese’s next film starring DiCaprio, De Niro
5 
 0.7793726 
 Apple reportedly nabs Scorcese film starring DiCaprio and De Niro
6 
 0.77775156 
 Apple lands Martin Scorsese movie Killers of the Flower Moon starring De Niro and DiCaprio
7 
 0.77384615 
 Martin Scorsese’s ‘Killers of the Flower Moon’ Starring Leonardo DiCaprio, Robert De Niro To Be An Apple Movie
8 
 0.7671393 
 Apple secures deal for Scorsese’s next film starring DiCaprio, De Niro: media
9 
 0.7671393 
 Apple secures deal for Scorsese's next film starring DiCaprio, De Niro -media
10 
 0.7671393 
 Apple secures deal for Scorsese's next film starring DiCaprio, De Niro: media


## 2. Cleans up and tokenizes article bodies using the RegexTokenizer and Stopword remover functions provided in the Class ExerciseDownload Class Exercise

In [29]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [31]:
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/20 18:41:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/20 18:41:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/20 18:41:16 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [32]:
# Enable Arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

## Loads previously deduplicated obtained dataset into a Spark dataframe

In [33]:
df = spark.createDataFrame(dedupdocu)

/Users/taylor/opt/anaconda3/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:425: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Nested StructType not supported in conversion from Arrow: struct<country: string, domain_rank: int64, main_image: string, participants_count: int64, performance_score: int64, published: string, replies_count: int64, section_title: string, site: string, site_categories: list<item: string>, site_full: string, site_section: string, site_type: string, social: struct<facebook: struct<comments: int64, likes: int64, shares: int64>, gplus: struct<shares: int64>, linkedin: struct<shares: int64>, pinterest: struct<shares: int64>, stumbledupon: struct<shares: int64>, vk: struct<shares: int64>>, spam_score: double, title: string, title_full: string, url: string, uuid: string>
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fa

In [34]:
df.printSchema()

root
 |-- Index: long (nullable = true)
 |-- thread: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- uuid: string (nullable = true)
 |-- url: string (nullable = true)
 |-- ord_in_thread: long (nullable = true)
 |-- parent_url: double (nullable = true)
 |-- author: string (nullable = true)
 |-- published: string (nullable = true)
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- highlightText: string (nullable = true)
 |-- highlightTitle: string (nullable = true)
 |-- highlightThreadTitle: string (nullable = true)
 |-- language: string (nullable = true)
 |-- external_links: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- external_images: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: string (valueContainsNull = true)
 |-- entities: map (nullable = true)
 |    |-- key: string
 |    |-- value: array (valueContai

In [35]:
df.select('title','text').show(20)

22/10/20 18:41:41 WARN TaskSetManager: Stage 0 contains a task of very large size (12222 KiB). The maximum recommended task size is 1000 KiB.


22/10/20 18:41:46 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 0 (TID 0): Attempting to kill Python Worker
+--------------------+--------------------+
|               title|                text|
+--------------------+--------------------+
|13 Reasons Why: T...|The controversial...|
|Judge gives contr...|A federal judge i...|
|A TV reboot of Bo...|WhatsApp If you e...|
|2-Pack: Ideaworks...|A Real Buzz-Worth...|
|Already-Obese Ave...|by Tyler Durden T...|
|Netflix, Disney j...|Hollywood is voic...|
|Novel Entertainme...|Novel Entertainme...|
|Anime Based On Be...|2 min read share\...|
|Tiger King star C...|Updated : 3 Jun 2...|
|All about Netflix...|All about Netflix...|
|File:Federation s...|A Federation stat...|
|news: Reemerged d...|QUEEN ELIZABETH I...|
|Samsung TV Plus L...|The niche LGBTQ+ ...|
|US Says Will Inve...|Washington, Jun 3...|
|COMPACT PRO G3 | ...|Display Technolog...|
|Even with Masterc...|Glenn Dyer's TV R...|
|Who Is Abigail Ko...|Who Is A

In [37]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, Word2Vec

In [38]:
# regular expression tokenizer to tokenize inputText into individual tokens (words)
regextok = RegexTokenizer(gaps = False, pattern = '\w+', 
                          inputCol = 'text', outputCol = 'tokens')
# StopWordsRemover to remove stopwords in the list of tokens
stopwrmv = StopWordsRemover(inputCol = 'tokens', outputCol = 'tokens_sw_removed')

df = regextok.transform(df)
df = stopwrmv.transform(df)

In [43]:
df.select('tokens_sw_removed','tokens','text').show(20)

22/10/21 01:52:50 WARN TaskSetManager: Stage 2 contains a task of very large size (12222 KiB). The maximum recommended task size is 1000 KiB.


22/10/21 01:52:54 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 2 (TID 2): Attempting to kill Python Worker
+--------------------+--------------------+--------------------+
|   tokens_sw_removed|              tokens|                text|
+--------------------+--------------------+--------------------+
|[controversial, 1...|[the, controversi...|The controversial...|
|[federal, judge, ...|[a, federal, judg...|A federal judge i...|
|[whatsapp, enjoye...|[whatsapp, if, yo...|WhatsApp If you e...|
|[real, buzz, wort...|[a, real, buzz, w...|A Real Buzz-Worth...|
|[tyler, durden, t...|[by, tyler, durde...|by Tyler Durden T...|
|[hollywood, voici...|[hollywood, is, v...|Hollywood is voic...|
|[novel, entertain...|[novel, entertain...|Novel Entertainme...|
|[2, min, read, sh...|[2, min, read, sh...|2 min read share\...|
|[updated, 3, jun,...|[updated, 3, jun,...|Updated : 3 Jun 2...|
|[netflix, sci, fi...|[all, about, netf...|All about Netflix...|
|[federation, stat...

## 3. Trains a Word2Vec model based on the output column produced in step 2. Implements any sample search query, which can be any combination of words of interest, and produces the top 10 matching article titles among all article titles. 

## Build up the model

In [51]:
word2vec = Word2Vec(vectorSize = 100, minCount = 3, inputCol = 'tokens_sw_removed', outputCol = 'wordvectors',seed=1, 
                   windowSize= 1)

In [52]:
model2 = word2vec.fit(df)

22/10/21 02:13:01 WARN TaskSetManager: Stage 13 contains a task of very large size (12222 KiB). The maximum recommended task size is 1000 KiB.


22/10/21 02:13:05 WARN TaskSetManager: Stage 15 contains a task of very large size (12222 KiB). The maximum recommended task size is 1000 KiB.


## Reload the dataset

In [53]:
df2 = spark.createDataFrame(dedupdocu)

In [54]:
df2 = regextok.transform(df2)
df2 = stopwrmv.transform(df2)
df2 = model2.transform(df2)

In [55]:
df2.count()

22/10/21 02:30:44 WARN TaskSetManager: Stage 18 contains a task of very large size (12222 KiB). The maximum recommended task size is 1000 KiB.


19513

In [101]:
chunks = df2.select('title','wordvectors').collect()

22/10/21 03:21:31 WARN TaskSetManager: Stage 47 contains a task of very large size (12222 KiB). The maximum recommended task size is 1000 KiB.


In [103]:
chunks[0]

Row(title="13 Reasons Why: The popular Netflix show's creator teases chance of a hopeful ending", wordvectors=DenseVector([0.0733, -0.0365, -0.0192, -0.0049, -0.0898, -0.0469, 0.0029, -0.0093, 0.0084, -0.0431, -0.0397, -0.1332, -0.0714, -0.117, 0.0003, 0.0742, 0.0286, 0.0355, 0.0543, -0.0168, -0.0051, -0.0149, -0.0326, 0.0291, -0.0325, -0.0434, 0.0509, -0.0473, 0.0637, -0.0398, 0.0206, 0.0923, 0.023, 0.0942, 0.0025, 0.0285, -0.0041, 0.0294, -0.0556, 0.0018, 0.0186, 0.0876, 0.0192, 0.0023, -0.0696, -0.1378, -0.0169, 0.051, -0.0152, 0.0017, 0.0951, 0.0881, -0.0551, -0.0363, -0.0587, -0.0044, 0.0273, -0.0185, -0.0237, -0.0588, -0.0161, -0.0195, -0.0555, 0.0765, 0.028, -0.0179, 0.0115, 0.0661, -0.0215, 0.1255, 0.0055, 0.053, -0.1649, 0.0028, -0.0036, -0.0213, 0.0683, -0.0873, -0.0541, 0.0059, -0.0469, 0.0192, -0.1632, 0.0051, 0.0362, 0.0126, 0.043, 0.0292, 0.0131, -0.0475, -0.1018, 0.0309, -0.0865, -0.0486, 0.0521, 0.0286, 0.0157, -0.0175, 0.1242, 0.0043]))

## Create a search query in Spark

In [87]:
def SEARCH_QUERY(text):
    query_df = spark.createDataFrame([(1, text)]).toDF('index','text')
    query_tok = regextok.transform(query_df)
    query_swr = stopwrmv.transform(query_tok)
    query_vec = model2.transform(query_swr)
    query_vec.show()
    query_vec = query_vec.select('wordvectors').collect()[0][0]
    return(query_vec)

In [114]:
query_vec=SEARCH_QUERY("I love NLP")

+-----+----------+--------------+-----------------+--------------------+
|index|      text|        tokens|tokens_sw_removed|         wordvectors|
+-----+----------+--------------+-----------------+--------------------+
|    1|I love NLP|[i, love, nlp]|      [love, nlp]|[0.09419919084757...|
+-----+----------+--------------+-----------------+--------------------+



### Calculate Cosine Similarity

In [92]:
import numpy as np
def cossim(v1, v2): 
    
    if np.dot(v1, v1) == 0 or np.dot(v2, v2) == 0:
        return 0.0
    return float(np.dot(v1, v2) / np.sqrt(np.dot(v1, v1)) / (np.sqrt(np.dot(v2, v2))))

In [115]:
data = [(i[0], float(cossim(query_vec, i[1]))) for i in chunks]
sim_df = spark.createDataFrame(data).toDF( 'title', 'similarity').orderBy('similarity', ascending=False)
sim_df.show(10, truncate = False)

+---------------------------------------------------------------------------------------------------+------------------+
|title                                                                                              |similarity        |
+---------------------------------------------------------------------------------------------------+------------------+
|Review: Netflix’s The Half of It queers a tired, age-old love story                                |0.6181124119098784|
|ClickTheCity: 'The Half Of It' On Netflix Movie Review                                             |0.6164820355289613|
|Netflix’s ‘The Half of It’ Tells a Story About All Kinds of Love                                   |0.6085601570181687|
|The Half Of It REVIEW – Coming-of-Age Excellence                                                   |0.595284412371208 |
|Why Netflix’s ‘A Secret Love’ Made Me Uncomfortable––And I’m Not The Only One                      |0.5829134552647822|
|Director Alice Wu On Her New Fi